In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine

raw_data = load_wine()
data = raw_data.data
feature_names = raw_data.feature_names
df = pd.DataFrame(data, columns=feature_names)
df['target'] = raw_data.target

In [2]:
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


Для решения задачи используется линейный классификатор, поэтому имеет смысл искать искомое подмножество признаков, основываясь на их множественной корреляции Пирсона с целевой переменной - высокая корреляция показывает, что классы хорошо линейно разделяются по целевому признаку. Будем перебирать всевозможные подмножества признаков и остановимся, когда множественная корреляция перестанет значительно расти при увеличении мощности подмножеств. Искомые признаки выберем из числа рассмотренных подмножеств наибольшей мощности, на которых достигается максимум множественной кореляции.

Кроме того, поскольку целевой признак - категориальный, чтобы исключить влияние искуственно введенного на нем порядка, будем рассматривать всевозможные перестановки его значений (т.е. перестановки чисел, соответствующих классам). Можно сузить число уникальных перестановок до 3ех, т.к. , например, (0, 1, 2) и (2, 1, 0) дадут равные по модулю коэффициенты множественной корреляции.

In [3]:
import scipy.stats as st
from itertools import chain, combinations

def target_permutation(perm, target):
    temp = np.copy(target)
    temp = np.where(temp==0, 3, temp)
    temp = np.where(temp==1, 4, temp)
    temp = np.where(temp==2, perm[2], temp)
    temp = np.where(temp==3, perm[0], temp)
    temp = np.where(temp==4, perm[1], temp)
    
    return temp


def powerset(iterable):

    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [4]:
def squared_mul_correlation(x, m):
    corr_matrix = np.zeros((len(m), len(m)))
    c = np.zeros(len(m))
    
    for i in range(len(m)):
        c[i] = st.pearsonr(x, np.argsort(np.argsort(df[m[i]])))[0]
        for j in range(len(m)):
            corr_matrix[i][j] = st.pearsonr(np.argsort(np.argsort(df[m[i]])),
                                            np.argsort(np.argsort(df[m[j]])))[0]
    R = np.linalg.inv(corr_matrix)
    return c @ R @ c.T

In [5]:
n_features = 0
squared_mul_corr = []
r = 0
target = df[df.columns[-1]]

for column_set in powerset(df.columns[:-1]):
    if len(column_set) > 0:
        for perm in ([0, 1, 2], [1, 0, 2], [2, 0, 1]):
            perm_target = target_permutation(perm, target)
            squared_mul_corr.append(squared_mul_correlation(perm_target, column_set))
        if n_features != len(column_set):
            n_features += 1
            if abs(r - max(squared_mul_corr)) < 5e-2:
                break
            else:
                r = max(squared_mul_corr)
                squared_mul_corr = []


In [6]:
squared_mul_corr = []
sets = []
for column_set in powerset(df.columns[:-1]):
    if len(column_set) == n_features:
        for perm in ([0, 1, 2], [1, 0, 2], [2, 0, 1]):
            perm_target = target_permutation(perm, target)
            squared_mul_corr.append(squared_mul_correlation(perm_target, column_set)) 
            sets.append(column_set)
important_features = list(sets[np.argmax(np.array(squared_mul_corr))])       

In [12]:
print("Наиболее важные признаки: {}".format(important_features))

Наиболее важные признаки: ['flavanoids', 'color_intensity', 'proline']


Сравним качество линейной классификации на полном множестве признаков и найденном нами подмножестве:

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


pipe = Pipeline([("scale", StandardScaler()), ("logreg", LogisticRegression())])
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
cross_val_score(pipe, raw_data.data, raw_data.target, cv=cv).mean()

0.9814814814814814

In [9]:
pipe = Pipeline([("scale", StandardScaler()), ("logreg", LogisticRegression())])
cross_val_score(pipe, df[important_features], raw_data.target, cv=cv).mean()

0.9444444444444443

Максимум качества классификации по всем подвыборкам признаков той же мощности:

In [10]:
scores = []
for column_set in powerset(df.columns[:-1]):
    if len(column_set) == n_features:
        pipe = Pipeline([("scale", StandardScaler()), ("logreg", LogisticRegression())])
        scores.append(cross_val_score(pipe, df[list(column_set)], raw_data.target, cv=cv).mean())
print("min: {},  mean: {},  max: {}".format(min(scores), sum(scores) / len(scores), max(scores)))

min: 0.5802469135802469,  mean: 0.8185919019252352,  max: 0.9629629629629629


Видно, что найденные признаки гораздо лучше разделяются по целевой переменной линейной гиперплоскостью, чем в среднем по всевозможным наборам из 3ех признаков.  